In [1]:
from GoogleNews import GoogleNews
from pprint import pprint
from newspaper import Article
import newspaper
import yaml
import datetime
import random
from pymongo import MongoClient
import string
from dateutil.relativedelta import relativedelta
import dateparser

In [2]:
terms = ['covid', 'coronavirus', 'koronavirus', 'virus']

In [6]:
news_links_list = []
website_by_url = {}
for term in terms:
    googlenews = GoogleNews()
    googlenews.setlang('tr')
    now = datetime.datetime.now()
    now = now.strftime("%m/%d/%Y")
    googlenews.setTimeRange('12/01/2019',now)
    googlenews.search(term)
    downloaded_links_count = len(googlenews.get__links())
    count_page = 2
    finished = False
    while not finished:
        googlenews.getpage(count_page)
        print(count_page)
        print(len(googlenews.get__links()))
        print('='*75)
        count_page += 1
        if len(googlenews.get__links()) > downloaded_links_count:
            downloaded_links_count = len(googlenews.get__links())
        else:
            finished = True
    for art in googlenews.result():
        website_by_url[art['link']] = art['media']
    news_links_list.append(googlenews.get__links())

2
20
3
30
4
40
5
50
6
60
7
70
8
80
9
90
10
100
11
110
12
120
13
130
14
140
15
150
16
160
17
170
18
180
19
190
20
200
21
210
22
220
23
230
24
240
25
250
26
260
27
270
28
280
29
290
30
300
31
310
32
320
33
330
34
338
35
340
36
340
2
20
3
20
2
20
3
30
4
40
5
50
6
60
7
70
8
80
9
90
10
100
11
110
12
120
13
130
14
140
15
150
16
160
17
170
18
180
19
190
20
190
2
20
3
30
4
40
5
50
6
60
7
70
8
80
9
90
10
100
11
110
12
120
13
130
14
140
15
150
16
160
17
160


In [7]:
def get_article_id(stringLength=86):
    lettersAndDigits = string.ascii_letters + string.digits + '-' + '_'
    return ''.join((random.choice(lettersAndDigits) for i in range(stringLength)))

In [8]:
def get_now():
    res = datetime.datetime.strftime(datetime.datetime.now(), '%Y-%m-%dT%H:%M:%S.%f')[:-3]
    #print(res)
    res = dateparser.parse(res).replace(tzinfo=None)
    #print(res)
    #res -= datetime.timedelta(days=1)
    #print(res)
    return res

In [19]:
client = MongoClient("mongodb://localhost:27021")
collection = client['news']['article_tr']

i = 0
for urls in news_links_list:
    for url in urls:
        print(i)
        print(url)
        if i >= 123:
            article = Article(url)
            article.download()
            article.parse()
            article.nlp()
            article_dict = {
                'id' : get_article_id(),
                'publishDate': article.publish_date,
                'discoverDate': get_now(),
                'title': article.title,
                'language': 'tr',
                'text': article.text,
                'structuredText': article.article_html,
                'url': article.url,
                'elements': list(article.imgs),
                'website': {
                    'name': website_by_url[url],
                },
                'metadata': list(article.tags),
                'highlight': '',
                'authors': article.authors,
                'keywords': article.keywords,
                'summary': article.summary

            }
            collection.insert_one(article_dict)
        i += 1
        #print(article.url, article.title, article.top_img, article.imgs, article.tags, article.authors, article.publish_date, article.text, article.article_html)
        #print(article.keywords, article.summary)

0
https://www.aa.com.tr/tr/dunya/dso-yeni-tip-koronavirusun-adini-covid-19-olarak-degistirdi/1731137
1
https://www.sozcu.com.tr/2020/saglik/turk-ilac-firmasi-covid-19-tedavisinde-onerilen-ilaci-uretmeye-basladi-5722770/
2
https://www.bbc.com/turkce/haberler-dunya-52041919
3
https://www.aa.com.tr/tr/dunya/covid-organics-madagaskar-halkini-ikiye-boldu/1821075
4
https://www.ntv.com.tr/galeri/dunya/rusya-covid-19-vakasinda-dunya-5inciligine-yerlesti,BPJkT6Hv50iZZyIlpenMlA
5
https://www.sozcu.com.tr/2020/saglik/bakanliktan-ofis-ve-isyerlerinde-covid-19a-karsi-alinacak-15-onlem-uyarisi-5736890/
6
https://www.sozcu.com.tr/2020/gundem/tubitak-baskani-mandal-covid-19-asisi-ve-ilaci-icin-tarih-verdi-5773328/
7
https://www.ntvspor.net/futbol/mke-ankaragucu-nde-covid-19-vakasi-5eb11b1e6e39a49f1c852016
8
https://www.sozcu.com.tr/2020/saglik/bilim-kurulu-uyesi-akcay-acikladi-covid-19-kis-hastaligi-mi-5689450/
9
https://www.trthaber.com/haber/dunya/dunya-genelinde-covid-19dan-olenlerin-sayisi-80-bini

124
https://www.birgun.net/haber/istanbul-il-saglik-mudurlugu-izin-almadan-covid-19-la-ilgili-bilimsel-calisma-yapilmasini-yasakladi-297385
125
https://www.birgun.net/haber/fitch-covid-19-ulkelerin-mali-durumlarini-zayiflatacak-292177
126
https://www.trthaber.com/haber/dunya/ab-ulkeleri-covid-19-kurtarma-paketinde-anlasti-474851.html
127
https://www.sabah.com.tr/ekonomi/2020/04/02/covid-19-kamu-ihale-sozlesmeleri-icin-mucbir-sebep-kabul-edilebilecek
128
https://www.sozcu.com.tr/2020/saglik/dr-ozer-covid-19-saglik-calisanlari-icin-meslek-hastaligi-ilan-edilmeli-5821040/
129
https://www.ntv.com.tr/dunya/fransada-okullar-acildiktan-sonra-70-ogrencide-covid-19-vakasi,co6Xcfxv2UGrbrx_HdUCQw
130
https://www.ntv.com.tr/dunya/ingilterede-kopekler-covid-19u-koklayarak-teshis-etmeleri-icin-egitilecek,x8gURyut90GA2zYmLRj7DQ
131
https://www.sozcu.com.tr/2020/gundem/akpli-vekilin-oglundan-skandal-covid-19-paylasimi-sozcuye-konusup-ozur-dilediler-5709971/
132
https://www.ntv.com.tr/turkiye/usakta-26

203
https://www.dw.com/tr/covid-19-ile-m%C3%BCcadele-vitaminsiz-olmuyor/a-53385773
204
https://www.birgun.net/haber/almanya-da-26-yasindaki-tc-vatandasi-covid-19-dan-hayatini-kaybetti-298317
205
https://www.trthaber.com/haber/dunya/misir-ve-fasta-covid-19-kaynakli-olum-ve-vaka-sayisi-artti-471029.html
206
https://www.sozcu.com.tr/2020/saglik/cocuklar-ve-ergenler-acisindan-covid-19-paneli-yapilacak-5819772/
207
http://bianet.org/bianet/saglik/223390-ttb-den-bakanliga-hekimin-olum-nedeninde-neden-covid-19-yazilmadi
208
https://www.fotomac.com.tr/haberler/2020/03/28/yeni-tip-corona-virusu-covid-19-pangolinden-mi-bulasti-pangolin-nedir
209
https://www.sozcu.com.tr/2020/saglik/corona-virusu-belirtileri-nelerdir-corona-virusu-baslangicta-nasil-belirtiler-gosterir-szcu2-5762710/
210
https://www.gazeteduvar.com.tr/spor/2020/05/22/yeni-malatyasporda-2-kisinin-covid-19-testi-pozitif-cikti/
211
https://www.hurriyet.com.tr/gundem/erzincanda-99-yasindaki-kadin-covid-19u-yendi-41505010
212
https://w

284
https://www.birgun.net/haber/polis-akademisi-nden-covid-19-raporu-aile-ici-siddet-artti-302114
285
https://www.birgun.net/haber/new-york-times-birinci-sayfasinda-covid-19-dan-olenlerin-isimleriyle-cikti-302113
286
https://ilkha.com/dunya/iran-da-58-kisi-daha-covid-19-nedeniyle-oldu-126103
287
https://tr.euronews.com/2020/05/24/af-orgutu-nden-cezaevindeki-covid-19-aff-ndan-yararlanamayan-dusunce-suclular-icin-cagr
288
https://www.trthaber.com/haber/dunya/sudanda-covid-19-kaynakli-vaka-ve-olumler-artiyor-487145.html
289
https://www.birgun.net/haber/ankara-tabip-odasi-ankara-da-362-saglik-calisanina-covid-19-tanisi-konuldu-302038
290
https://www.birgun.net/haber/zambiya-enformasyon-bakani-covid-19-a-yakalandi-302138
291
https://www.trthaber.com/haber/dunya/belcikada-covid-19-vaka-sayisi-57-bini-gecti-487282.html
292
https://www.gazeteduvar.com.tr/dunya/2020/05/25/oxford-covid-19-asisinda-iki-faza-gecti-10-bin-260-kiside-denenecek/
293
https://www.haberturk.com/samsun-haberleri/7816634

361
https://www.formsante.com.tr/turkiye-gunluk-koronavirus-tablosu-canli-guncel-vaka-ve-olu-sayisi/
362
https://www.hurriyet.com.tr/galeri-corona-virus-turkiye-son-durum-tablosu-koronavirus-turkiye-vaka-ve-olum-sayilari-ile-en-guncel-korona-covid-19-haritasi-41518070
363
https://www.bbc.com/turkce/haberler-turkiye-51832978
364
https://www.milliyet.com.tr/koronavirus-nereden-geldi-nasil-ortaya-cikti-salgin-dunyaya-yayiliyor-mu--molatik-14011/
365
https://www.haberturk.com/aciklandi-17-mayis-koronavirus-tablosu-koronavirusten-bugun-kac-kisi-oldu-corona-virus-vaka-sayisi-turkiye-2682730
366
https://www.hurriyet.com.tr/galeri-corona-virusu-son-durum-3-mayis-dunyada-ve-turkiyede-koronavirus-korona-virus-vaka-ve-olum-sayisi-tablosu-41505520
367
https://www.aa.com.tr/tr/dunya/cinde-yeni-tip-koronavirus-salgini-nedeniyle-can-kaybi-304e-cikti/1721591
368
https://www.aa.com.tr/tr/dunya/cin-de-yeni-koronavirus-salginindan-olenlerin-sayisi-909-a-yukseldi/1729087
369
https://www.aa.com.tr/tr/anali

441
https://www.bbc.com/turkce/haberler-dunya-51381099
442
https://www.aa.com.tr/tr/analiz/koronavirus-kuresel-ekonominin-yeni-siyah-kugu-su-mu/1731812
443
https://www.ahaber.com.tr/gundem/2020/05/22/bilim-kurulu-uyesi-prof-dr-levent-yamanelden-koronavirus-aciklamasi-yasak-gelmeseydi
444
https://www.aa.com.tr/tr/dunya/iranin-kum-kentinde-koronavirus-nedeniyle-turbe-ziyaretine-sinirlama/1743403
445
https://www.haberturk.com/abd-koronavirus-ile-mucadele-icin-rusya-ya-solunum-cihazi-bagislayacak-2686509
446
https://www.sabah.com.tr/galeri/turkiye/son-dakika-koronavirus-anketinden-carpici-sonuclar-yuzde-90lik-oran
447
https://tr.sputniknews.com/koronavirus-salgini/202005161042057307-koronavirusten-son-24-saatte-41-kisi-hayatini-kaybetti-/
448
https://www.bbc.com/turkce/haberler-dunya-52767844
449
https://finans.mynet.com/haber/detay/ekonomi/koronavirus-doneminde-kandira-daki-arazilere-talep-patladi/395488/
450
https://www.hurriyet.com.tr/kelebek/hurriyet-cumartesi/nezaketiyle-sosyal-medyad

516
https://www.aa.com.tr/tr/yasam/koronaviruse-uyarladigi-sarkilarla-evde-kalanlari-eglendiriyor/1775614
517
https://www.sabah.com.tr/galeri/magazin/burak-akkul-bir-an-once-olsun-su-virus-koronavirus-tedavisi-gormustu
518
https://www.fanatik.com.tr/besiktastan-koronavirus-aciklamasi-2144165
519
https://www.fotomac.com.tr/genel/2020/05/13/d-vitamini-eksikligi-corona-virusunden-koronavirus-olum-ihtimalini-artiriyor-mu-iste-yapilan-o-arastirma
520
https://www.hurriyet.com.tr/ekonomi/koronavirus-salgini-cicek-satislarini-artirdi-41476372
521
https://www.hurriyet.com.tr/dunya/abdde-son-24-saatte-koronavirus-kaynakli-bin-68-olum-41521291
522
https://www.haberturk.com/abd-baskani-trump-tan-koronavirus-asi-calismalari-icin-isik-hizindan-daha-hizli-proje-haberler-2681036
523
https://www.aa.com.tr/tr/yasam/koronaviruse-uyarladigi-sarkilarla-evde-kalanlari-eglendiriyor/1775614
524
https://www.sabah.com.tr/galeri/magazin/burak-akkul-bir-an-once-olsun-su-virus-koronavirus-tedavisi-gormustu
525
htt

588
https://www.hurriyet.com.tr/dunya/tunusta-corona-virus-kaynakli-5inci-olum-41478268
589
https://www.hurriyet.com.tr/galeri-corona-virus-son-durum-turkiye-kac-vaka-var-ve-olum-sayisi-nedir-istanbul-ilce-ilce-covid-19-risk-haritasi-41514665
590
https://www.hurriyet.com.tr/yazarlar/osman-muftuoglu/virus-guc-mu-kaybetti-41520608
591
https://www.hurriyet.com.tr/gundem/corona-virus-nedeniyle-olenlerin-sayisi-cezayirde-10a-fasta-3e-yukseldi-41473790
592
https://www.sozcu.com.tr/2020/ekonomi/youtube-corona-virus-videolarinda-reklami-kaldirdi-5664535/
593
https://www.hurriyet.com.tr/ekonomi/art-arda-duyuruyor-unlu-is-insanindan-corona-virus-yardimi-41470736
594
https://www.milliyet.com.tr/gundem/corona-virus-belirtileri-neler-korunma-yollari-nedir-yeni-belirti-aciklandi-iste-corona-virusun-gun-gun-belirtileri-6185941
595
https://www.bbc.com/turkce/haberler-dunya-51171231
596
https://www.ntvspor.net/futbol/virus-zlatan-a-gelmezse-zlatan-viruse-gider-5e7233f9474561178020ef7f
597
https://www.s

661
https://www.sabah.com.tr/saglik/2020/04/08/nukseden-corona-virus-vakalarinin-sirri-cozuldu-corona-virus-hucrelerde-gizlice-bekliyor-olabilir
662
https://www.sabah.com.tr/galeri/saglik/bilim-kurulu-acikladi-iste-corona-virus-ile-ilgili-merak-edilenler
663
https://webrazzi.com/2020/03/04/havayolu-ucus-coronavirus-should-you-fly/
664
https://www.sozcu.com.tr/2020/ekonomi/altin-fiyatlarinda-virus-etkisi-gram-ve-ceyrek-altin-fiyatlari-ne-kadar-5600767/
665
https://www.sabah.com.tr/saglik/2020/04/08/corona-virus-ile-ilgili-onemli-uyari-corona-virusu-onlemek-icin-sirke-yerine
666
https://www.sabah.com.tr/galeri/saglik/arastirmanin-detaylari-ortaya-cikti-corona-virus-havada-asili-mi-kaliyor-iste-cevabi
667
https://www.sabah.com.tr/galeri/dunya/son-dakika-haberi-dunyada-corona-virus-vaka-ve-olu-sayisi-kac-oldu-iste-ulke-ulke-corona-virus-raporu
668
https://www.sabah.com.tr/galeri/saglik/damacanada-korkutan-corona-virus-iddiasi-tam-72-saat-boyunca
669
https://www.ntv.com.tr/galeri/dunya/coro